In [ ]:
from datasets import load_dataset
import matplotlib

Get data

In [ ]:
data_files = {
    'train': 'train.jsonl',
    'test': 'test.jsonl'
}

dataset = load_dataset('json', data_files=data_files)
train_dataset = dataset['train']
test_dataset = dataset['test']